### Dependencies

In [53]:
# !pip install keybert

In [1]:
# !pip install keybert[flair]
# !pip install keybert[gensim]
# !pip install keybert[spacy]
# !pip install keybert[use]

In [2]:
# !pip install spacy

In [213]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from statistics import mean
from keybert import KeyBERT

### Data Cleaning

In [214]:
cleaned_data = pd.read_csv('./CleanedDataset.csv') # This is an updated version

In [215]:
cleaned_data.head()

,last_updated,tweet_id,created_at,full_text,processed_tweet,Party,quote_count,reply_count,retweet_count,Importance,...,Neutral,Positive,Negative,Compound,hashtags,user_mentions_screen_name,City,State,Country,Score
0,2019-02-08 17:29:00,1.090000e+18,2019-02-08 17:28:00,@DasComrade Sir any comment on modi's grand en...,dascomrad sir comment modi grand entri,BJP,0.0,0.0,1.0,0,...,0.625,0.375,0.000,0.4588,NaN,DasComrade,Guwahati,Assam,India,0.045645
1,2019-02-08 17:29:00,1.090000e+18,2019-02-07 05:10:00,Congress made Vemula national talking point ev...,congress made vemula nation talk point even th...,BJP,47.0,72.0,2353.0,1,...,0.868,0.000,0.132,-0.5719,NaN,NaN,Mumbai,Maharashtra,India,108.990410
2,2019-02-08 17:30:00,1.090000e+18,2019-02-08 10:18:00,BJP has been winning battles over the past mon...,bjp win battl past month bengal pm modi arriv ...,BJP,1.0,12.0,513.0,1,...,0.582,0.276,0.142,0.5719,NaN,NaN,Kolkata,West Bengal,India,24.487725
3,2019-02-08 17:30:00,1.090000e+18,2019-02-08 17:29:00,@inclusivemind2 @Ark82876233 @BJP4India @Rahul...,inclusivemind ark bjpindia rahul gandhi bjp th...,Other,0.0,0.0,2.0,0,...,0.884,0.116,0.000,0.4404,NaN,"inclusivemind2,Ark82876233,BJP4India,RahulGandhi",Aligarh,Uttar Pradesh,India,0.091290
4,2019-02-08 17:30:00,1.090000e+18,2019-02-08 14:19:00,Senior Congress leader @ManishTewari : There w...,senior congress leader manishtewari jpc defini...,Congress,0.0,1.0,159.0,1,...,1.000,0.000,0.000,0.0000,NaN,ManishTewari,New Delhi,Delhi,India,8.263217


In [216]:
list(cleaned_data.columns)

['last_updated',
 'tweet_id',
 'created_at',
 'full_text',
 'processed_tweet',
 'Party',
 'quote_count',
 'reply_count',
 'retweet_count',
 'Importance',
 'favorite_count',
 'Neutral',
 'Positive',
 'Negative',
 'Compound',
 'hashtags',
 'user_mentions_screen_name',
 'City',
 'State',
 'Country',
 'Score']

In [217]:
data = cleaned_data.drop(['tweet_id','Neutral','Positive','Negative','City','State','Country','user_mentions_screen_name'],axis=1)

In [218]:
data.head()

,last_updated,created_at,full_text,processed_tweet,Party,quote_count,reply_count,retweet_count,Importance,favorite_count,Compound,hashtags,Score
0,2019-02-08 17:29:00,2019-02-08 17:28:00,@DasComrade Sir any comment on modi's grand en...,dascomrad sir comment modi grand entri,BJP,0.0,0.0,1.0,0,1,0.4588,NaN,0.045645
1,2019-02-08 17:29:00,2019-02-07 05:10:00,Congress made Vemula national talking point ev...,congress made vemula nation talk point even th...,BJP,47.0,72.0,2353.0,1,2353,-0.5719,NaN,108.990410
2,2019-02-08 17:30:00,2019-02-08 10:18:00,BJP has been winning battles over the past mon...,bjp win battl past month bengal pm modi arriv ...,BJP,1.0,12.0,513.0,1,513,0.5719,NaN,24.487725
3,2019-02-08 17:30:00,2019-02-08 17:29:00,@inclusivemind2 @Ark82876233 @BJP4India @Rahul...,inclusivemind ark bjpindia rahul gandhi bjp th...,Other,0.0,0.0,2.0,0,2,0.4404,NaN,0.091290
4,2019-02-08 17:30:00,2019-02-08 14:19:00,Senior Congress leader @ManishTewari : There w...,senior congress leader manishtewari jpc defini...,Congress,0.0,1.0,159.0,1,159,0.0000,NaN,8.263217


In [219]:
data.head()

,last_updated,created_at,full_text,processed_tweet,Party,quote_count,reply_count,retweet_count,Importance,favorite_count,Compound,hashtags,Score
0,2019-02-08 17:29:00,2019-02-08 17:28:00,@DasComrade Sir any comment on modi's grand en...,dascomrad sir comment modi grand entri,BJP,0.0,0.0,1.0,0,1,0.4588,NaN,0.045645
1,2019-02-08 17:29:00,2019-02-07 05:10:00,Congress made Vemula national talking point ev...,congress made vemula nation talk point even th...,BJP,47.0,72.0,2353.0,1,2353,-0.5719,NaN,108.990410
2,2019-02-08 17:30:00,2019-02-08 10:18:00,BJP has been winning battles over the past mon...,bjp win battl past month bengal pm modi arriv ...,BJP,1.0,12.0,513.0,1,513,0.5719,NaN,24.487725
3,2019-02-08 17:30:00,2019-02-08 17:29:00,@inclusivemind2 @Ark82876233 @BJP4India @Rahul...,inclusivemind ark bjpindia rahul gandhi bjp th...,Other,0.0,0.0,2.0,0,2,0.4404,NaN,0.091290
4,2019-02-08 17:30:00,2019-02-08 14:19:00,Senior Congress leader @ManishTewari : There w...,senior congress leader manishtewari jpc defini...,Congress,0.0,1.0,159.0,1,159,0.0000,NaN,8.263217


In [220]:
data.fillna(0,inplace=True)

In [221]:
data.head()

,last_updated,created_at,full_text,processed_tweet,Party,quote_count,reply_count,retweet_count,Importance,favorite_count,Compound,hashtags,Score
0,2019-02-08 17:29:00,2019-02-08 17:28:00,@DasComrade Sir any comment on modi's grand en...,dascomrad sir comment modi grand entri,BJP,0.0,0.0,1.0,0,1,0.4588,0,0.045645
1,2019-02-08 17:29:00,2019-02-07 05:10:00,Congress made Vemula national talking point ev...,congress made vemula nation talk point even th...,BJP,47.0,72.0,2353.0,1,2353,-0.5719,0,108.990410
2,2019-02-08 17:30:00,2019-02-08 10:18:00,BJP has been winning battles over the past mon...,bjp win battl past month bengal pm modi arriv ...,BJP,1.0,12.0,513.0,1,513,0.5719,0,24.487725
3,2019-02-08 17:30:00,2019-02-08 17:29:00,@inclusivemind2 @Ark82876233 @BJP4India @Rahul...,inclusivemind ark bjpindia rahul gandhi bjp th...,Other,0.0,0.0,2.0,0,2,0.4404,0,0.091290
4,2019-02-08 17:30:00,2019-02-08 14:19:00,Senior Congress leader @ManishTewari : There w...,senior congress leader manishtewari jpc defini...,Congress,0.0,1.0,159.0,1,159,0.0000,0,8.263217


In [206]:
data_full_text = pd.DataFrame(data['full_text'])

In [207]:
data_full_text

,full_text
0,@DasComrade Sir any comment on modi's grand en...
1,Congress made Vemula national talking point ev...
2,BJP has been winning battles over the past mon...
3,@inclusivemind2 @Ark82876233 @BJP4India @Rahul...
4,Senior Congress leader @ManishTewari : There w...
...,...
39869,A web page dedicated to tracking Rahul Gandhi'...
39870,Itâ€™s time to end the forever wars. But in th...
39871,The only phone number you need to reach any re...
39872,"Bjp officer bearer, faked Assangeâ€™s endorsem..."


In [222]:
data.columns

Index(['last_updated', 'created_at', 'full_text', 'processed_tweet', 'Party',
       'quote_count', 'reply_count', 'retweet_count', 'Importance',
       'favorite_count', 'Compound', 'hashtags', 'Score'],
      dtype='object')

In [223]:
len(data)

39874

In [224]:
data = data[data['Compound']<0]

In [225]:
data

,last_updated,created_at,full_text,processed_tweet,Party,quote_count,reply_count,retweet_count,Importance,favorite_count,Compound,hashtags,Score
1,2019-02-08 17:29:00,2019-02-07 05:10:00,Congress made Vemula national talking point ev...,congress made vemula nation talk point even th...,BJP,47.0,72.0,2353.0,1,2353,-0.5719,0,108.990410
6,2019-02-08 17:30:00,2019-02-08 09:16:00,#RahulLieCaught | EXCLUSIVE: Former Defence Se...,rahulliecaught exclus former defenc secretari ...,Congress,22.0,34.0,680.0,1,680,-0.2960,RahulLieCaught,32.315419
8,2019-02-08 17:30:00,2019-02-08 16:30:00,These stories really exposes our development p...,stori realli expos develop propaganda nation c...,BJP,1.0,0.0,10.0,0,10,-0.2500,0,0.460270
9,2019-02-08 17:30:00,2019-02-08 08:57:00,"If there is no issue or problem, why is Parrik...",issu problem parrikar say defenc secretari res...,BJP,3.0,3.0,123.0,1,123,-0.4019,0,6.642802
17,2019-02-08 17:30:00,2019-02-07 16:14:00,Prime Minister Shri @narendramodi today expose...,prime minist shri modi today expos congress pa...,BJP,52.0,295.0,10761.0,1,10761,-0.4767,0,494.056771
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39862,2019-05-16 18:28:00,2019-05-16 15:27:00,.@fayedsouza to BJP's @ShainaNC: You've refuse...,fayedsouza bjps shainanc refus rehous repeat w...,BJP,1.0,3.0,18.0,0,18,-0.7184,0,0.842442
39865,2019-05-16 18:28:00,2019-05-16 11:48:00,@chitrapadhi Paper thin skin of BJP leaders......,chitrapadhi paper thin skin bjp leadersand hin...,BJP,0.0,0.0,3.0,0,3,-0.5653,0,0.136935
39868,2019-05-16 18:28:00,2019-05-16 11:43:00,This is way way way more important than what h...,way way way import happen decad ago rahul gand...,Congress,0.0,0.0,10.0,0,10,-0.8271,0,0.456449
39869,2019-05-16 18:29:00,2019-05-16 14:55:00,A web page dedicated to tracking Rahul Gandhi'...,web page dedic track rahul gandhi numer lie ma...,Congress,0.0,1.0,18.0,0,18,-0.3887,0,0.827280


In [226]:
len(data)

14119

In [252]:
data.to_csv('./data.csv')

In [253]:
data = pd.read_csv('./data.csv')

In [254]:
data

,Unnamed: 0,last_updated,created_at,full_text,processed_tweet,Party,quote_count,reply_count,retweet_count,Importance,favorite_count,Compound,hashtags,Score
0,1,2019-02-08 17:29:00,2019-02-07 05:10:00,Congress made Vemula national talking point ev...,congress made vemula nation talk point even th...,BJP,47.0,72.0,2353.0,1,2353,-0.5719,0,108.990410
1,6,2019-02-08 17:30:00,2019-02-08 09:16:00,#RahulLieCaught | EXCLUSIVE: Former Defence Se...,rahulliecaught exclus former defenc secretari ...,Congress,22.0,34.0,680.0,1,680,-0.2960,RahulLieCaught,32.315419
2,8,2019-02-08 17:30:00,2019-02-08 16:30:00,These stories really exposes our development p...,stori realli expos develop propaganda nation c...,BJP,1.0,0.0,10.0,0,10,-0.2500,0,0.460270
3,9,2019-02-08 17:30:00,2019-02-08 08:57:00,"If there is no issue or problem, why is Parrik...",issu problem parrikar say defenc secretari res...,BJP,3.0,3.0,123.0,1,123,-0.4019,0,6.642802
4,17,2019-02-08 17:30:00,2019-02-07 16:14:00,Prime Minister Shri @narendramodi today expose...,prime minist shri modi today expos congress pa...,BJP,52.0,295.0,10761.0,1,10761,-0.4767,0,494.056771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14114,39862,2019-05-16 18:28:00,2019-05-16 15:27:00,.@fayedsouza to BJP's @ShainaNC: You've refuse...,fayedsouza bjps shainanc refus rehous repeat w...,BJP,1.0,3.0,18.0,0,18,-0.7184,0,0.842442
14115,39865,2019-05-16 18:28:00,2019-05-16 11:48:00,@chitrapadhi Paper thin skin of BJP leaders......,chitrapadhi paper thin skin bjp leadersand hin...,BJP,0.0,0.0,3.0,0,3,-0.5653,0,0.136935
14116,39868,2019-05-16 18:28:00,2019-05-16 11:43:00,This is way way way more important than what h...,way way way import happen decad ago rahul gand...,Congress,0.0,0.0,10.0,0,10,-0.8271,0,0.456449
14117,39869,2019-05-16 18:29:00,2019-05-16 14:55:00,A web page dedicated to tracking Rahul Gandhi'...,web page dedic track rahul gandhi numer lie ma...,Congress,0.0,1.0,18.0,0,18,-0.3887,0,0.827280


In [255]:
data['full_text']

0        Congress made Vemula national talking point ev...
1        #RahulLieCaught | EXCLUSIVE: Former Defence Se...
2        These stories really exposes our development p...
3        If there is no issue or problem, why is Parrik...
4        Prime Minister Shri @narendramodi today expose...
                               ...                        
14114    .@fayedsouza to BJP's @ShainaNC: You've refuse...
14115    @chitrapadhi Paper thin skin of BJP leaders......
14116    This is way way way more important than what h...
14117    A web page dedicated to tracking Rahul Gandhi'...
14118    Itâ€™s time to end the forever wars. But in th...
Name: full_text, Length: 14119, dtype: object

In [256]:
data_full_text = pd.DataFrame(data['full_text'])

In [257]:
data_full_text

,full_text
0,Congress made Vemula national talking point ev...
1,#RahulLieCaught | EXCLUSIVE: Former Defence Se...
2,These stories really exposes our development p...
3,"If there is no issue or problem, why is Parrik..."
4,Prime Minister Shri @narendramodi today expose...
...,...
14114,.@fayedsouza to BJP's @ShainaNC: You've refuse...
14115,@chitrapadhi Paper thin skin of BJP leaders......
14116,This is way way way more important than what h...
14117,A web page dedicated to tracking Rahul Gandhi'...


In [258]:
data_full_text.to_csv('.data_full_text.csv')

In [259]:
len(data)

14119

In [260]:
len(data_full_text)

14119

In [261]:
data_full_text['full_text'][12001]

'@fire_on_ice82 @arunjaitley BJP has sided against poor home buyers #SAVEJPWISHTOWN @ndtv @TheHinduWeekend @TimesNow #ElectionsWithNDTV #ElectionsWithTimes'

### Triggers fetched using Key Bert

In [262]:
triggers = pd.read_csv('./5000triggers.csv')

In [263]:
triggers

,Unnamed: 0,0,1,2,3,4
0,0,dalit discourse,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN
3,3,defence secretary,NaN,NaN,NaN,NaN
4,4,fake campaign,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5893,5893,terrorist cong,terrorists pakis,congis different,NaN,NaN
5894,5894,gave bjp,bjp,bjp an,shame rahullovesterrorists,NaN
5895,5895,pakistan modioncemore,NaN,NaN,NaN,NaN
5896,5896,demonetisation disaster,demonetisationyaadrakhna,NaN,NaN,NaN


In [264]:
triggers.drop('Unnamed: 0',axis=1,inplace=True)

In [265]:
triggers_list = [[y for y in x if pd.notna(y)] for x in triggers.values.tolist()]

In [266]:
triggers_list

[['dalit discourse'],
 [],
 [],
 ['defence secretary'],
 ['fake campaign'],
 ['modi shah', 'rafalescam saugataroymp', 'shah'],
 ['mahathugbandhan',
  'mahathugbandhan is',
  'mahabhrashtbandhan mahathugbandhan'],
 [],
 [],
 ['parrikar admits', 'admits pmo'],
 ['case tharoor'],
 ['kerala rss', 'kerala', 'sewa bharathi', 'over kerala', 'kerala for'],
 ['rahulgandhi dear', 'dear rahul', 'rahul gandhi'],
 ['gandhi based', 'hindu clearly', 'gandhi'],
 ['rafaledeal decision', 'of rafaledeal'],
 ['rahul attacks', 'are rahul'],
 ['crore rahulgandhi',
  'rahulgandhi pakdagayamodi',
  'pm modi',
  'modi is',
  'stolen countryâ'],
 ['cancel rafaledeal', 'rafaledeal and'],
 [],
 ['modi govt'],
 ['exists rafalecongselfgoal', 'rafalecongselfgoal'],
 [],
 [],
 ['constitutional institutions'],
 ['racist sweat'],
 ['merabharat', 'merabharat most'],
 ['are whitaker', 'whitaker'],
 ['the chowkidar'],
 ['bail cancellation', 'manage judges', 'judges'],
 ['modi is', 'modi', 'is coward', 'coward'],
 ['angelt

In [267]:
triggers.fillna(0,inplace = True)

In [268]:
triggers

,0,1,2,3,4
0,dalit discourse,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,defence secretary,0,0,0,0
4,fake campaign,0,0,0,0
...,...,...,...,...,...
5893,terrorist cong,terrorists pakis,congis different,0,0
5894,gave bjp,bjp,bjp an,shame rahullovesterrorists,0
5895,pakistan modioncemore,0,0,0,0
5896,demonetisation disaster,demonetisationyaadrakhna,0,0,0


In [269]:
triggers.head()

,0,1,2,3,4
0,dalit discourse,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,defence secretary,0,0,0,0
4,fake campaign,0,0,0,0


In [270]:
len(triggers)

5898

In [271]:
triggers.to_csv('./FinalTriggers.csv')

In [ ]:
## you have got trigger_list and trigger

### Use of text Moderator API

In [128]:
sent = "I hate Mangoes"
url = "https://text-moderator.p.rapidapi.com/api/v1/moderate"
payload = { "input": sent }
headers = {
    "content-type": "application/json",
    "X-RapidAPI-Key": "a64eedac50mshd91efa213f0c2a7p1901dcjsnaf67556610b7",
    "X-RapidAPI-Host": "text-moderator.p.rapidapi.com"
}
response = requests.post(url, json=payload, headers=headers)
print(response.json())


{'toxic': 0.37751234, 'indecent': 0.039553843, 'threat': 0.0104988385, 'offensive': 0.13607834, 'erotic': 0.024652457, 'spam': 0.08515842}


In [272]:
import requests
def CheckHate(sent):
    url = "https://text-moderator.p.rapidapi.com/api/v1/moderate"
    payload = { "input": sent }
    headers = {
        "content-type": "application/json",
        "X-RapidAPI-Key": "a64eedac50mshd91efa213f0c2a7p1901dcjsnaf67556610b7",
        "X-RapidAPI-Host": "text-moderator.p.rapidapi.com"
    }
    response = requests.post(url, json=payload, headers=headers)
    # print(response.json())
    doc1 = response.json().values()
    
    if(mean(doc1)>=0.1):
        return True
    else:
        return False

In [273]:
print(CheckHate('dalit discourse'))

False


In [131]:
print(CheckHate('terrorist cong'))

True


In [132]:
print(CheckHate('cpi violence'))

False


In [274]:
triggers.head()

,0,1,2,3,4
0,dalit discourse,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,defence secretary,0,0,0,0
4,fake campaign,0,0,0,0


In [276]:
triggers_list

[['dalit discourse'],
 [],
 [],
 ['defence secretary'],
 ['fake campaign'],
 ['modi shah', 'rafalescam saugataroymp', 'shah'],
 ['mahathugbandhan',
  'mahathugbandhan is',
  'mahabhrashtbandhan mahathugbandhan'],
 [],
 [],
 ['parrikar admits', 'admits pmo'],
 ['case tharoor'],
 ['kerala rss', 'kerala', 'sewa bharathi', 'over kerala', 'kerala for'],
 ['rahulgandhi dear', 'dear rahul', 'rahul gandhi'],
 ['gandhi based', 'hindu clearly', 'gandhi'],
 ['rafaledeal decision', 'of rafaledeal'],
 ['rahul attacks', 'are rahul'],
 ['crore rahulgandhi',
  'rahulgandhi pakdagayamodi',
  'pm modi',
  'modi is',
  'stolen countryâ'],
 ['cancel rafaledeal', 'rafaledeal and'],
 [],
 ['modi govt'],
 ['exists rafalecongselfgoal', 'rafalecongselfgoal'],
 [],
 [],
 ['constitutional institutions'],
 ['racist sweat'],
 ['merabharat', 'merabharat most'],
 ['are whitaker', 'whitaker'],
 ['the chowkidar'],
 ['bail cancellation', 'manage judges', 'judges'],
 ['modi is', 'modi', 'is coward', 'coward'],
 ['angelt

In [277]:
len(triggers_list)

5898

### To extract list of triggers

In [ ]:
## We 
i=0
final_trigger_list=[]
for list in triggers_list:
    new_list=[]
    for word in list:
        if (CheckHate(word)==True):
           new_list.append(word) 
    if len(new_list)!=0:
        final_trigger_list.append(new_list)
    

* We have fed this list to the API and checked that the triggers generated are hate or no Hate
* So now we have finally only hate triggers

In [49]:
len(final)

0

In [52]:
final_trigger_list # Out of 5900 only 498-500 could be checked for hate using Text Moderator API

[['stop bjp'],
 ['rahul gandhi', 'rahul', 'tear rahul', 'rahulliecaught', 'mohan kumar'],
 ['stori jharkhand', 'jharkhand govt'],
 ['secretari resolv',
  'defenc secretari',
  'problemmatt princip',
  'princip secretari'],
 ['shri modi'],
 ['rafalescam saugataroymp', 'modishah duo'],
 ['everi crook', 'forthefirsttim everi'],
 ['terrorist threat', 'indian media'],
 ['fake scam', 'boycottthehindu banthehindu', 'scam without'],
 ['blame modi'],
 ['complaint shashitharoor', 'delhi court'],
 ['kerala poor', 'hous kerala', 'kerala rss', 'kerala', 'bharathi build'],
 ['rahul gandhi', 'gandhi dear', 'dear rahul'],
 ['hindu clear', 'gandhi', 'rahul gandhi'],
 ['govt submit', 'interfer govt', 'govt rafal', 'sc govt', 'process govt'],
 ['rshivshankar rafalecongselfgo',
  'rahul attack',
  'rafal withstand',
  'modi rafal',
  'govt rafal'],
 ['modi thief',
  'thief stolen',
  'stolen countri',
  'gandhi pakdagayamodi',
  'thief'],
 ['cancel rafaled'],
 [],
 ['modi govt'],
 ['exist rafalecongselfgo

### Incorporating Key Bert in a function

In [278]:
def KeyBert(doc,i):
    from keybert import KeyBERT
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(doc)
    return kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, i), stop_words=None)

In [279]:
import spacy

In [280]:
nlp = spacy.load("en_core_web_md")

### Basic functions

In [281]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [282]:
def find_most_similar_triggers(input_sentence):
    similarity_dict = {}
    input_sent_keywords = KeyBert(input_sentence, 2)

    input_sent_keywords = [i[0] for i in input_sent_keywords]
    sent1 = str(input_sent_keywords)
    answer = 0
    index = -1
    for ind, sent in enumerate(triggers_list):
        m = len(sent)
        if m == 0:
            similarity_dict[ind] = 0
            continue
        sent2 = str(sent)
        vector1 = text_to_vector(sent1)
        vector2 = text_to_vector(sent2)
        cosine_similarity = get_cosine(vector1, vector2)
        if cosine_similarity>answer:
            index = ind
            answer= cosine_similarity
    return index

 

### Hashtag Extraction Functions

In [283]:
df_ht = data[['favorite_count', 'Compound', 'hashtags']]
df_ht

,favorite_count,Compound,hashtags
0,2353,-0.5719,0
1,680,-0.2960,RahulLieCaught
2,10,-0.2500,0
3,123,-0.4019,0
4,10761,-0.4767,0
...,...,...,...
14114,18,-0.7184,0
14115,3,-0.5653,0
14116,10,-0.8271,0
14117,18,-0.3887,0


In [284]:
df_ht = df_ht[df_ht['Compound'] < 0]
df_ht.reset_index(inplace = True, drop = True)


df_ht.iloc[1, 2]

hashtag_favorite_count_dict = {}


for i in range(len(df_ht)):
    if df_ht['hashtags'][i] is not np.nan:
        hashtags = df_ht['hashtags'][i].split(',')
        for hashtag in hashtags:
            if hashtag not in hashtag_favorite_count_dict.keys():
                hashtag_favorite_count_dict[hashtag] = 0
            hashtag_favorite_count_dict[hashtag] += df_ht['favorite_count'][i]

In [285]:
hashtag_favorite_count_dict

{'0': 5681140,
 'RahulLieCaught': 680,
 'RafaleScam': 1916,
 'LokSabha': 49,
 'ForTheFirstTime': 349,
 'NaMo': 49,
 'Rafale': 4691,
 'FakeNews': 66,
 'RafaleDeal': 12890,
 'Delhi': 106,
 'FIR': 15,
 'Kerala': 1386,
 'DEAR': 20,
 'PakdaGayaModi': 952,
 'MeraBharat': 0,
 'Whitaker': 4,
 'StartUpIndia': 94,
 'AngelTax': 94,
 'à¤²à¥‹à¤•à¤¤à¤‚à¤¤à¥\x8dà¤°vsà¤\xadà¥€à¤¡à¤¼à¤¤à¤‚à¤¤à¥\x8dà¤°': 19,
 'GOP': 436,
 'WhitakerHearing': 2,
 'Benghazi': 1,
 'contempt': 1,
 'CropLikeTheHindu': 181,
 'PakdagayaModi': 66,
 'chaiwala': 21,
 'NSSO': 35,
 'pappu': 0,
 'Ramalingam': 851,
 'ModiUnstoppable': 34,
 'WorldCancerDay': 34,
 'CampusFaceOff': 115,
 'Goa': 3,
 'ManoharParrikar': 5019,
 'BJP': 11994,
 'GoBackModi': 12881,
 'TNWelcomesModi': 435,
 'DMKFails': 42,
 'farmers': 11,
 'agriculture': 11,
 'GoBackSadistModi': 180,
 'Asuras': 1,
 'law': 12,
 'India': 4735,
 'Brahmins': 12,
 'religions': 12,
 'castes': 12,
 'HR1': 72,
 'PoorPeoplesCampaign': 9,
 'Congress': 3969,
 'Modi': 23096,
 'shame': 2,
 

In [286]:
hashtag_favorite_count_list = sorted(hashtag_favorite_count_dict.items(), key = lambda x:x[1], reverse = True)
hashtag_favorite_count_list 

[('0', 5681140),
 ('Gurugram', 44805),
 ('GreenNewDeal', 33404),
 ('Modi', 23096),
 ('ProtectTheVA', 20915),
 ('PulwamaAttack', 19706),
 ('RafaleDeal', 12890),
 ('GoBackModi', 12881),
 ('RSS', 12221),
 ('BJP', 11994),
 ('RajivGandhi', 11521),
 ('Pulwama', 10006),
 ('RahulGandhi', 9607),
 ('NiravModi', 8526),
 ('WATCH', 8448),
 ('ChowkidarChorHai', 8012),
 ('SadhviPragya', 7334),
 ('HowsTheJosh', 6850),
 ('MasoodAzhar', 6482),
 ('ModiOnZee', 6364),
 ('HolocaustRemembranceDay', 5633),
 ('PulwamaTerrorAttack', 5584),
 ('MainBhiChowkidar', 5581),
 ('AtishiMarlena', 5166),
 ('ManoharParrikar', 5019),
 ('MyFirstVoteForModi', 5013),
 ('LokSabhaElections2019', 4997),
 ('CorruptModi', 4991),
 ('Pakistan', 4985),
 ('EarthDay', 4975),
 ('JallianwalaBagh', 4931),
 ('Telangana', 4905),
 ('CongressMuktBharat', 4825),
 ('IndianAirForce', 4743),
 ('India', 4735),
 ('Rafale', 4691),
 ('UNSC', 4167),
 ('BhagRahulBhag', 4160),
 ('Surgicalstrike2', 4083),
 ('FileChorChowkidar', 4021),
 ('Congress', 3969),

### Model Implementation

In [287]:
input_sentence = data_full_text['full_text'][12002]

In [288]:
print(input_sentence)

@pawanjhawat Leaders of @BJP4India n Ministers of #NDA_ModiGovt including @PMOIndia ~ Vat wud u say nw on #HighFuelPrices ven Crude oil prices in International Market is much lower dan vat it was during #UPARegime (100 -140 USD/Barrel in UPA period vs
40 - 80 USD/Barrel)?

#HeightOfHypocrisy


In [289]:
similarity_list = find_most_similar_triggers(input_sentence)

In [290]:
similarity_list

3236

In [291]:
triggers.iloc[similarity_list]

0       indiaâ crude
1    reducing indiaâ
2        oil imports
3          crude oil
4                  0
Name: 3236, dtype: object

In [292]:
print(input_sentence)

@pawanjhawat Leaders of @BJP4India n Ministers of #NDA_ModiGovt including @PMOIndia ~ Vat wud u say nw on #HighFuelPrices ven Crude oil prices in International Market is much lower dan vat it was during #UPARegime (100 -140 USD/Barrel in UPA period vs
40 - 80 USD/Barrel)?

#HeightOfHypocrisy


In [293]:
print(data['full_text'][similarity_list])

Prime Minister Narendra Modi sets out a road map for reducing Indiaâ€™s crude oil imports by 10 per cent by 2022

@shinejac 

https://t.co/ONtw8bFPQ2


In [294]:
input_sent_keywords = KeyBert(input_sentence, 2)
input_sent_keywords = [i[0] for i in input_sent_keywords]

In [295]:
input_sent_keywords

['oil prices',
 'on highfuelprices',
 'highfuelprices ven',
 'highfuelprices',
 'crude oil']

In [296]:
for i in input_sent_keywords:
    triggers_extracted = []
    if CheckHate(i) == True:
        triggers_extracted.append(i)

print(triggers_extracted)

[]


In [297]:
def final_model(input_sentence):
    print("Your Input Sentence : "+input_sentence)
    print("------------------------------")
    print("HashTags:")
    import re
    # Use a regular expression to find words starting with '#'
    hashtags = re.findall(r'#\w+', input_sentence)
    print(hashtags)
    print("------------------------------")
    similarity_list = find_most_similar_triggers(input_sentence) # returns index
    print("Past Triggers : ")
    print(triggers.iloc[similarity_list])
    print(data['full_text'][similarity_list])
    print("------------------------------")
    input_sent_keywords = KeyBert(input_sentence, 2)
    input_sent_keywords = [i[0] for i in input_sent_keywords]
    for i in input_sent_keywords:
        triggers_extracted = []
        if CheckHate(i) == True:
            triggers_extracted.append(i)
    
    print("Hate Speech Generated by input sentence")
    print(triggers_extracted)

In [307]:
final_model(data_full_text['full_text'][12143])

Your Input Sentence : Did @facebook know the CHAIRMAN of @AtlanticCouncil owned the #ShadowNet when they partnered to restore integrity to elections worldwide?

Will journalists report this? Is the world ok with such deceit? Did Mark Zuckerberg LIE TO CONGRESS @annvandersteel?

https://t.co/hNa5uvOcbU
------------------------------
HashTags:
['#ShadowNet']
------------------------------
Past Triggers : 
0           zuckerberg lied
1                zuckerberg
2           mark zuckerberg
3    facebook whistleblower
4                         0
Name: 3771, dtype: object
Shocking testimony from the Facebook whistleblower reveals that Mark Zuckerberg LIED to Congress https://t.co/0EpX0gg2Zi
------------------------------
Hate Speech Generated by input sentence
[]
